# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Загружаем библиотеки.

In [ ]:
#загрузка библиотек
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Скачаем датасет и сохраним в переменную

In [ ]:
#сохранение датасета в переменной
df = pd.read_csv("/datasets/insurance.csv")

Выведем датасет и проверим информацию.

In [ ]:
#вывод датасета
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [ ]:
#вывод информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Изменим типы данных в столбцах.

In [ ]:
#изменение типов данных
df = df.astype(int)

Изменим язык и регистр названий столбцов

In [ ]:
#изменение языка и регистра названий столбцов
df = df.rename(columns  = {"Пол":"gender", 
                      "Возраст":"age", 
                      "Зарплата":"income", 
                      "Члены семьи":"family", 
                      "Страховые выплаты":"payments"})

Далее разделим датафрейм на признаки и целевой признак.

In [ ]:
#разделение на признаки
features = df.drop(["payments"], axis = 1 )
target = df["payments"]

## Умножение матриц

### Теоретическая часть

Для работы нам необходимы несколько формул.


Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Задачу обучения можно записать формулой:

$$
w = (X^T X)^{-1} X^T y
$$

Для предсказания нам потребуется формула:

$$
a = Xw+w_0
$$


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w_0$ - сдвиг

Так же у нас будет преобразование признаков и слeдовательно будет формула:

$$
a_1 = X_1w_1 + w_0
$$ 



**Задача: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

Создадим обритимую(квадратную), рандомную матрицу $P$ размером равным количеству столбцов в таблице признаков.

Т.к при создании матрицы может быть так что матрица будет необратимая(содержать 0). Напишем функцию для создания обратимой матрицы.  Функция поможет наверняка избежать необратимости.

Преобразуем признаки с помощью матрицы и проверим размер новой таблицы.

In [ ]:
#размер таблицы признаков
print(features.shape)

#создадим матрицу
P = rnd_matrix(features.shape[1])
print(P.shape)

#применим матричное умножение
features_new = features @ P 
print(features_new.shape)

#выведем новые признаки
features_new.head()               

(5000, 4)
(4, 4)
(5000, 4)


,0,1,2,3
0,-66711.478891,-5219.915102,61130.484784,-82561.195585
1,-51113.638283,-4018.173722,46817.791548,-63248.413034
2,-28247.325332,-2225.582133,25869.267869,-34952.361052
3,-56083.817426,-4369.434206,51409.159560,-69414.745854
4,-35105.329026,-2755.910727,32160.139103,-43442.835113


После умножения признаков на рандомную матрицу у нас образовались новые признаки.  
Запишем это формулой:

$$
X_1 = XP
$$

При преобразовании признаков мы видим что значения меняются но размер таблицы остаётся тем же.

Получившимися новыми признаками обучим модель линейной регрессии.  
Метрикой R2 проверим нашу модель.

In [ ]:
#обучение модели
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)

print(r2_score(target, predictions))

0.42494550308113277


**Ответ:**  
Видим что коэффициент детерминации этой модели равен 42%. 

**Обоснование:**  
В процессе трансформации признаков была применена формула преобразования: $X_1 = XP$  
Где $X$ - признаки, $P$ - матрица шифрования.  
При этом можно заметить что при разных значениях $P$ результат предсказа R2 сильно не меняется. Из этого можно сдельать вывод что при шифровании значение $P$ не влияет на результат. В этом процессе происходит только сокрытие данных $X$

## Алгоритм преобразования

**Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.**

**Алгоритм**

Создадим класс линейной регрессии который содержит 2 метода: обучение модели и предсказание.  
Метод обучения принимает признаки и целевой признак. 
Далее к матрице признаков добавляется нулевой столбец куда записываются все единицы.
Вектор весов находится по формуле:

$$
w = (X^T X)^{-1} X^T y
$$

Затем сохраняем вектор весов как параметр функции при этом нулевое значение вектора это отлельный пареметр сдвига.



Метод пердсказания работает по формуле:

$$
a = Xw + w_0
$$ 



In [ ]:
#класс линейной регрессии
class LinReg:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

**Обоснование**  
Цель данной задачи была доказать что качество линейной регрессии не меняется при преобразованных признаках и исходных.

$$
a = a_1
$$

Если для неизменённых данных действует обычная формула, то для преобразованных данных работает формула:

$$
a_1 = X_1w_1 + w_0
$$ 
Где:

$$
X_1 = XP
$$

Где:

$$
w_1 = (X_1^T X_1)^{-1} X_1^T y = ((XP)^TXP)^{-1}(XP)^Ty = 
$$
$$
(P^TX^TXP)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

В данной фофрмуле произведение матриц $(P^T)^{-1}P^T$  представляют собой единичную матрицу $E$. А так как в ходе умножения матриц $AE$ или  $EA$  результатом будет матрица $A$, то уберем единичную матрицу из формулы.

Получим формулу:

$$
w_1 = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w
$$

Из этого следует что:

$$
a_1 = X_1w_1 = (XP)(P^{-1}w) = XPP^{-1}w
$$

Опять матрицы $PP^{-1}$ образовали матрицу $E$.
Далее:

$$
a = Xw = a_1
$$

Следовательно матрицы  $a_1$ и $a$ тождественны.

$$
a = a_1
$$

Можно сделать вывод что предсказания $a$ и $a_1$ должны  давать одинаковый результат R2.

## Проверка алгоритма

Применим получившийся класс и предскажем результат.
Используем неизменённые признаки.

Сначала создадим модель из библиотеки sklearn и выведем результат.

In [ ]:
#модель линейной регрессии
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

print("R2:",r2_score(target, predictions))


R2: 0.42494550308169177


Обучим собственную модель.

In [ ]:
#модель линейной регрессии 
def_model = LinReg()
def_model.fit(features, target)
predictions = def_model.predict(features)

print("R2:",r2_score(target, predictions))

R2: 0.42494550308169177


Используем изменённые признаки.

In [ ]:
#модель линейной регрессии
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)

print("R2:",r2_score(target, predictions))

R2: 0.42494550308113277


In [ ]:
#модель линейной регрессии
def_model = LinReg()
def_model.fit(features_new,target)
predictions = def_model.predict(features_new)

print("R2:",r2_score(target,predictions))

R2: 0.42479009308923255


**Общие выводы**

Применяя библиотечную функцию и собственную, к данным исходным и преобразованным получили результат в 42%.
Т.к. результат идентичный, подтверждаем что при шифровании данных не влияет на качество линейной регрессии.
И следовательно:

$$
a = a_1
$$